<a href="https://colab.research.google.com/github/srudkin12/RegionalBallMapper/blob/main/Session_4_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Session 4**

Session 4 of the Topological Data Analysis Ball Mapper for Regional Analysis workshop is designed to be an informal look at some of the further functionality of BallMapper and a chance for participants to have a look at their own data. 

This notebook contains the results on regressions and the visualisation of residuals using BallMapper. Again you are encouraged to use this notebook in conjunction with the commentary file.

Because this is a separate Colab notebook we need to install the packages and read in the data. The following block of code completes these steps.

In [ ]:
install.packages("dplyr")
install.packages("BallMapper")
install.packages("car")
library(dplyr)
library(BallMapper)
library(car)

url = 'https://raw.githubusercontent.com/srudkin12/RegionalBallMapper/main/region1.csv'
dtx<-read.table(url,sep=",",header=TRUE)


We now create our subset of variables as in Session 3

In [ ]:
dty<-cbind(dtx[,1:2],dtx$QualLevel4,dtx$Deprivation0,dtx$Accommodation,dtx$Married,dtx$HealthVeryGood,dtx$OwnedMortgage)
names(dty)<-c("geog","geogcode","QualLevel4","Deprivation0","Accommodation","Married","HealthVeryGood","OwnedMortgage")
dty$QL4<-as.numeric(dty$QualLevel4>33) # Areas where more than 1/3rd have Degrees

In many regional datasets there are issues with strong correlations between variables. Therefore we must perform checks for multicollinearity

We first check for correlations of 0.7 or above

In [ ]:
cor(dty[,3:9])

The highest absolute correlation is -0.660 between OwnedMortgage and Accommodation. This does not exceed the 0.7 threshold in absolute value

The second approach is to use the variance inflation function. If you are unsure of the role of the VIF then please consult statistics books or web resources. For our purposes we are interested in whether there is a VIF above 10

In [ ]:
lm1<-lm(QualLevel4~Deprivation0+Accommodation+Married+HealthVeryGood+OwnedMortgage,data=dty)
vif(lm1)

The highest VIF is well below the 10 threshold and so we will continue to use the lm1 model.

We can produce a summary of the model using the following line of code:

In [ ]:
summary(lm1)

We will now use the fitted() function to create a new column of fitted values from the model lm1. This is stored as fit1. 

In [ ]:
dty$fit1<-fitted(lm1)

We may create a column of residuals by subtracting the fitted value from the true value of QualLevel4

In [ ]:
dty$res1<-dty$QualLevel4-dty$fit1

Now we have two extra outcomes to colour our BallMapper plot. The following code is built from the block in Session 3, but also includes the two new outputs. Notice that we also create the characteristics as a separate data.frame and then normalise using the function in the BallMapper package

In [ ]:
y1<-as.data.frame(dty$QualLevel4)
y2<-as.data.frame(dty$QL4)
y3<-as.data.frame(dty$fit1)
y4<-as.data.frame(dty$res1)

x1<-as.data.frame(dty[,4:8])

x2<-normalize_to_min_0_max_1(x1)

Let us now produce three BallMapper plots. One of the true value (t), one of the fitted values (f) and one of the residuals (r). Here we continue with the 0.3 radius identified in Session 3.

Note that the code does not actually plot the BallMapper graphs so there will be no output

In [ ]:
bm130t<-BallMapper(x2,y1,0.3)
bm130f<-BallMapper(x2,y3,0.3) 
bm130r<-BallMapper(x2,y4,0.3)  

We may plot the BallMapper graphs using the plot function. Again to see the others the code may be slightly edited to replace the letter after bm130

In [ ]:
ColorIgraphPlot(bm130r,seed_for_plotting=123)

An interesting result emerges here because the residuals are all positive on the edges of the graph, the main body has residuals near 0 as would be expected. The only strong negative residual appears in the outlier

We will now bring in the user defined function points_to_balls and follow the same steps as in Session 3 to identify the local authority districts in ball 16

In [ ]:
dty$pt<-seq(1:nrow(dty))

points_to_balls<-function(l){
  a001<-length(l$landmarks)
  a1<-matrix(0,nrow=a001,ncol=2)
  a1<-as.data.frame(a1)
  names(a1)<-c("pt","ball")
  for(i in 1:a001){
    a<-as.data.frame(l$points_covered_by_landmarks[i])
    names(a)<-"pt"
    a$ball<-i
    a1<-rbind.data.frame(a1,a)
   }
  a1<-a1[2:nrow(a1),]
  return(a1)
}

bmp1<-points_to_balls(bm130r)
bmp1<-as.data.frame(bmp1)
names(bmp1)<-c("pt","ball")

dtz<-merge(dty,bmp1,by="pt")

dtz16<-subset(dtz,dtz$ball==16)
dtz16

We can see that the region where the model overpredicts is the Isles of Scilly. Here the low OwnedMortgage would suggest a higher QualLevel4 than is observed given the negative coefficient on OwnedMortgage in the model. 

There are many more things that we may wish to do with modelling and considering residuals. For example we may look at binary classification models for the QL4 dummy. This workshop is intended to provide the basic building blocks for using Topological Data Analysis Ball Mapper.